In [ ]:
# Investigate music channel scaling and search for meaningful values
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd
import numpy as np

print("🔍 INVESTIGATING MUSIC CHANNEL SCALING")
print("=" * 50)

# Load genMusic01 data
bids_path = BIDSPath(
    subject='01',
    task='genMusic01', 
    datatype='eeg',
    root='./datasets/bcmi/bcmi-fmri'
)

raw = read_raw_bids(bids_path, verbose=False)

# Get the full music channel data
music_idx = raw.ch_names.index('music')
music_data = raw.get_data(picks=[music_idx])[0]

print(f"Music channel statistics:")
print(f"  Min: {music_data.min():.6e}")
print(f"  Max: {music_data.max():.6e}")
print(f"  Mean: {music_data.mean():.6e}")
print(f"  Std: {music_data.std():.6e}")

# Look for meaningful values (not tiny)
non_zero = music_data[music_data != 0]
print(f"  Non-zero values: {len(non_zero)}")

if len(non_zero) > 0:
    print(f"  Non-zero min: {non_zero.min():.6e}")
    print(f"  Non-zero max: {non_zero.max():.6e}")
    
    # Try different scaling factors
    print(f"\nTrying different scaling factors:")
    for scale in [1, 10, 20, 100, 1000, 10000, 100000, 1000000]:
        scaled_values = non_zero * scale
        reasonable_codes = scaled_values[(scaled_values >= 100) & (scaled_values <= 999)]
        
        print(f"  Scale x{scale:7d}: Range [{scaled_values.min():.1f}, {scaled_values.max():.1f}], "
              f"3-digit codes: {len(reasonable_codes)}")
        
        if len(reasonable_codes) > 0:
            print(f"    Sample 3-digit codes: {reasonable_codes[:5].astype(int)}")

# Check if there are distinct levels in the music channel
unique_values = np.unique(music_data)
print(f"\nUnique values in music channel: {len(unique_values)}")
if len(unique_values) <= 20:
    print(f"All unique values: {unique_values}")
else:
    print(f"First 10 unique values: {unique_values[:10]}")
    print(f"Last 10 unique values: {unique_values[-10:]}")

# Try to find the pattern around meaningful transitions
if len(unique_values) > 2:  # More than just 0 and one other value
    # Find transitions between different values
    diff_indices = np.where(np.diff(music_data) != 0)[0]
    print(f"\nTransitions in music channel: {len(diff_indices)}")
    
    if len(diff_indices) > 0:
        print("Sample transitions:")
        for i, idx in enumerate(diff_indices[:10]):
            time_s = idx / raw.info['sfreq']
            val_before = music_data[idx]
            val_after = music_data[idx + 1]
            print(f"  {i+1:2d}: t={time_s:7.3f}s, {val_before:.6e} -> {val_after:.6e}")

🔍 INVESTIGATING MUSIC CHANNEL SCALING
Music channel statistics:
  Min: 0.000000e+00
  Max: 4.115000e-05
  Mean: 3.410357e-07
  Std: 3.113802e-06
  Non-zero values: 11000
  Non-zero min: 1.405009e-05
  Non-zero max: 4.115000e-05

Trying different scaling factors:
  Scale x      1: Range [0.0, 0.0], 3-digit codes: 0
  Scale x     10: Range [0.0, 0.0], 3-digit codes: 0
  Scale x     20: Range [0.0, 0.0], 3-digit codes: 0
  Scale x    100: Range [0.0, 0.0], 3-digit codes: 0
  Scale x   1000: Range [0.0, 0.0], 3-digit codes: 0
  Scale x  10000: Range [0.1, 0.4], 3-digit codes: 0
  Scale x 100000: Range [1.4, 4.1], 3-digit codes: 0
  Scale x1000000: Range [14.1, 41.1], 3-digit codes: 0

Unique values in music channel: 8
All unique values: [0.00000000e+00 1.40500862e-05 1.40996910e-05 1.41499237e-05
 2.25996918e-05 4.10501625e-05 4.10997673e-05 4.11500000e-05]

Transitions in music channel: 22
Sample transitions:
   1: t=190.999s, 0.000000e+00 -> 1.405009e-05
   2: t=191.999s, 1.405009e-05 ->

/tmp/ipykernel_147882/2173051345.py:17: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_147882/2173051345.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_147882/2173051345.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_147882/2173051345.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_147882/2173051345.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of

In [12]:
# Map distinct music channel values to music codes
print("🎵 MAPPING MUSIC CHANNEL VALUES TO CODES")
print("=" * 50)

# We found 8 unique non-zero values
unique_music_vals = np.array([1.40500862e-05, 1.40996910e-05, 1.41499237e-05,
                              2.25996918e-05, 4.10501625e-05, 4.10997673e-05, 4.11500000e-05])

print("Distinct music channel values (non-zero):")
for i, val in enumerate(unique_music_vals):
    # Try different scaling approaches
    scaled_20 = val * 20
    scaled_100k = val * 100000
    scaled_1m = val * 1000000
    
    print(f"  Value {i+1}: {val:.6e}")
    print(f"    * 20 = {scaled_20:.6f}")
    print(f"    * 100k = {scaled_100k:.1f}")
    print(f"    * 1M = {scaled_1m:.1f}")
    print()

# Let's look at the actual pattern - maybe the values correspond to simple indices
print("Trying to map values to simple indices:")
value_to_code = {}
for i, val in enumerate(sorted(unique_music_vals)):
    code = i + 1  # Start from 1
    value_to_code[val] = code
    print(f"  {val:.6e} -> Code {code}")

print(f"\nValue-to-code mapping: {len(value_to_code)} codes")

# Now let's apply this mapping to the trial onsets
events_path = bids_path.copy().update(suffix='events', extension='.tsv')
events_df = pd.read_csv(events_path.fpath, sep='\t')
trial_starts = events_df[events_df['trial_type'] == 768].copy()
onsets = trial_starts['onset'].values
unique_onsets = onsets[np.concatenate([[True], np.diff(onsets) > 1])]

print(f"\nApplying mapping to trial onsets:")
music_idx = raw.ch_names.index('music')
sfreq = raw.info['sfreq']

for i, onset in enumerate(unique_onsets):
    sample_idx = int(onset * sfreq)
    if sample_idx < raw.n_times:
        music_val = raw.get_data(picks=[music_idx])[0, sample_idx]
        
        # Find closest matching value
        if music_val == 0:
            music_code = 0
            music_file = "generated/silence_or_baseline.wav"
        else:
            # Find the closest non-zero value
            closest_val = min(unique_music_vals, key=lambda x: abs(x - music_val))
            music_code = value_to_code[closest_val]
            
            # Format as traditional music file naming
            if music_code <= 9:
                # Single digit codes
                music_file = f"generated/unknown_code_{music_code}.wav"
            else:
                # Try to format as XXX -> X-X_X
                s_code = str(music_code).zfill(3)
                music_file = f"generated/{s_code[0]}-{s_code[1]}_{s_code[2]}.wav"
            
        print(f"  Trial {i+1:2d}: onset={onset:7.3f}s, val={music_val:.6e}, "
              f"code={music_code:2d}, file={music_file}")

# Let's also try a different approach - maybe the values encode actual music file numbers
print(f"\nAlternative: Direct value encoding (trying different scales):")
for i, onset in enumerate(unique_onsets[:5]):  # Just first 5 trials
    sample_idx = int(onset * sfreq)
    if sample_idx < raw.n_times:
        music_val = raw.get_data(picks=[music_idx])[0, sample_idx]
        
        # Try interpreting the value differently
        # Maybe it's encoded as a fraction or needs different scaling
        alt_code1 = int(round(music_val * 1000000))  # Scale to get integer
        alt_code2 = int(round(music_val / min(unique_music_vals)))  # Relative to smallest
        
        print(f"  Trial {i+1}: val={music_val:.6e}, alt1={alt_code1}, alt2={alt_code2}")

🎵 MAPPING MUSIC CHANNEL VALUES TO CODES
Distinct music channel values (non-zero):
  Value 1: 1.405009e-05
    * 20 = 0.000281
    * 100k = 1.4
    * 1M = 14.1

  Value 2: 1.409969e-05
    * 20 = 0.000282
    * 100k = 1.4
    * 1M = 14.1

  Value 3: 1.414992e-05
    * 20 = 0.000283
    * 100k = 1.4
    * 1M = 14.1

  Value 4: 2.259969e-05
    * 20 = 0.000452
    * 100k = 2.3
    * 1M = 22.6

  Value 5: 4.105016e-05
    * 20 = 0.000821
    * 100k = 4.1
    * 1M = 41.1

  Value 6: 4.109977e-05
    * 20 = 0.000822
    * 100k = 4.1
    * 1M = 41.1

  Value 7: 4.115000e-05
    * 20 = 0.000823
    * 100k = 4.1
    * 1M = 41.1

Trying to map values to simple indices:
  1.405009e-05 -> Code 1
  1.409969e-05 -> Code 2
  1.414992e-05 -> Code 3
  2.259969e-05 -> Code 4
  4.105016e-05 -> Code 5
  4.109977e-05 -> Code 6
  4.115000e-05 -> Code 7

Value-to-code mapping: 7 codes

Applying mapping to trial onsets:
  Trial  1: onset=191.151s, val=1.405009e-05, code= 1, file=generated/unknown_code_1.wav
 

In [13]:
# Test classical music trial processing
print("🎼 TESTING CLASSICAL MUSIC TRIALS")
print("=" * 40)

# Load classicalMusic data
classical_bids_path = BIDSPath(
    subject='01',
    task='classicalMusic', 
    datatype='eeg',
    root='./datasets/bcmi/bcmi-fmri'
)

try:
    classical_raw = read_raw_bids(classical_bids_path, verbose=False)
    classical_events_path = classical_bids_path.copy().update(suffix='events', extension='.tsv')
    classical_events_df = pd.read_csv(classical_events_path.fpath, sep='\t')
    
    print(f"Classical music recording:")
    print(f"  Duration: {classical_raw.times[-1]:.1f} seconds")
    print(f"  Channels: {classical_raw.info['nchan']}")
    print(f"  Sampling rate: {classical_raw.info['sfreq']} Hz")
    
    # Find trial start markers (768)
    classical_trial_starts = classical_events_df[classical_events_df['trial_type'] == 768]
    print(f"\nTrial start events (768): {len(classical_trial_starts)}")
    
    if len(classical_trial_starts) > 0:
        print("Trial start events:")
        print(classical_trial_starts[['onset', 'duration', 'trial_type']].head())
        
        # Get unique onsets (removing consecutive duplicates)
        classical_onsets = classical_trial_starts['onset'].values
        classical_unique_onsets = classical_onsets[np.concatenate([[True], np.diff(classical_onsets) > 1])]
        
        print(f"\nUnique trial onsets: {len(classical_unique_onsets)}")
        for i, onset in enumerate(classical_unique_onsets):
            duration_to_end = classical_raw.times[-1] - onset
            print(f"  Trial {i+1}: onset={onset:.3f}s, duration_to_end={duration_to_end:.1f}s")
        
        # Check if there's a music channel in classical music data
        if 'music' in classical_raw.ch_names:
            print(f"\n'music' channel found in classical music data")
            music_idx = classical_raw.ch_names.index('music')
            music_data = classical_raw.get_data(picks=[music_idx])[0]
            
            print(f"  Music channel stats:")
            print(f"    Min: {music_data.min():.6e}")
            print(f"    Max: {music_data.max():.6e}")
            print(f"    Unique values: {len(np.unique(music_data))}")
            
            # Sample at first trial onset
            if len(classical_unique_onsets) > 0:
                first_onset = classical_unique_onsets[0]
                sample_idx = int(first_onset * classical_raw.info['sfreq'])
                music_val = music_data[sample_idx]
                print(f"    Value at first trial onset: {music_val:.6e}")
        else:
            print(f"\nNo 'music' channel in classical music data")
            print(f"Available channels: {classical_raw.ch_names}")
    
except Exception as e:
    print(f"Error loading classical music data: {e}")

print(f"\n✅ Classical music test completed!")

🎼 TESTING CLASSICAL MUSIC TRIALS
Classical music recording:
  Duration: 1915.0 seconds
  Channels: 47
  Sampling rate: 1000.0 Hz

Trial start events (768): 12
Trial start events:
       onset  duration  trial_type
35    31.771     0.001         768
246  195.807     0.001         768
456  362.010     0.001         768
654  517.864     0.001         768
842  676.140     0.001         768

Unique trial onsets: 12
  Trial 1: onset=31.771s, duration_to_end=1883.2s
  Trial 2: onset=195.807s, duration_to_end=1719.2s
  Trial 3: onset=362.010s, duration_to_end=1553.0s
  Trial 4: onset=517.864s, duration_to_end=1397.1s
  Trial 5: onset=676.140s, duration_to_end=1238.9s
  Trial 6: onset=841.794s, duration_to_end=1073.2s
  Trial 7: onset=973.926s, duration_to_end=941.1s
  Trial 8: onset=1140.532s, duration_to_end=774.5s
  Trial 9: onset=1296.737s, duration_to_end=618.3s
  Trial 10: onset=1460.071s, duration_to_end=454.9s
  Trial 11: onset=1594.423s, duration_to_end=320.6s
  Trial 12: onset=1758.15

/tmp/ipykernel_147882/1370543555.py:14: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  classical_raw = read_raw_bids(classical_bids_path, verbose=False)
/tmp/ipykernel_147882/1370543555.py:14: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  classical_raw = read_raw_bids(classical_bids_path, verbose=False)
/tmp/ipykernel_147882/1370543555.py:14: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  classical_raw = read_raw_bids(classical_bids_path, verbose=False)
/tmp/ipykernel_147882/1370543555.py:14: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  classical_raw = read_raw_bids(classical_bids_path, verbose=False)
/tmp/ipykernel_147882/1370543555.py:14: 

  Music channel stats:
    Min: 0.000000e+00
    Max: 3.500000e-07
    Unique values: 5
    Value at first trial onset: 3.500000e-07

✅ Classical music test completed!


In [6]:
# Analyze classical music channel values in detail
print("🔍 CLASSICAL MUSIC CHANNEL ANALYSIS")
print("=" * 40)

music_idx = classical_raw.ch_names.index('music')
music_data = classical_raw.get_data(picks=[music_idx])[0]

# Find unique non-zero values
unique_values = np.unique(music_data)
non_zero_values = unique_values[unique_values > 0]

print(f"Unique music channel values in classical music:")
print(f"  Total unique values: {len(unique_values)}")
print(f"  Non-zero values: {len(non_zero_values)}")

for i, val in enumerate(non_zero_values):
    print(f"    Value {i+1}: {val:.10e}")

# Check values at each trial onset
print(f"\nMusic values at trial onsets:")
for i, onset in enumerate(classical_unique_onsets):
    sample_idx = int(onset * classical_raw.info['sfreq'])
    music_val = music_data[sample_idx]
    
    # Check if this corresponds to any known value
    if music_val > 0:
        value_idx = np.where(non_zero_values == music_val)[0]
        if len(value_idx) > 0:
            value_code = value_idx[0] + 1  # 1-based coding
        else:
            value_code = "unknown"
    else:
        value_code = 0
    
    print(f"  Trial {i+1} (t={onset:.3f}s): {music_val:.10e} -> code {value_code}")

# For classical music, we expect all trials to have the same music file
# The implementation suggests using run name for classical music rather than music channel
print(f"\nClassical music run type: task-classicalMusic")
print("Expected behavior: All trials should reference the same classical music file")
print("The music channel might not be meaningful for classical music trials")

🔍 CLASSICAL MUSIC CHANNEL ANALYSIS
Unique music channel values in classical music:
  Total unique values: 5
  Non-zero values: 4
    Value 1: 9.9998474098e-08
    Value 2: 1.4999771115e-07
    Value 3: 2.5000152590e-07
    Value 4: 3.5000000000e-07

Music values at trial onsets:
  Trial 1 (t=31.771s): 3.5000000000e-07 -> code 4
  Trial 2 (t=195.807s): 2.5000152590e-07 -> code 3
  Trial 3 (t=362.010s): 1.4999771115e-07 -> code 2
  Trial 4 (t=517.864s): 1.4999771115e-07 -> code 2
  Trial 5 (t=676.140s): 2.5000152590e-07 -> code 3
  Trial 6 (t=841.794s): 9.9998474098e-08 -> code 1
  Trial 7 (t=973.926s): 0.0000000000e+00 -> code 0
  Trial 8 (t=1140.532s): 0.0000000000e+00 -> code 0
  Trial 9 (t=1296.737s): 3.5000000000e-07 -> code 4
  Trial 10 (t=1460.071s): 9.9998474098e-08 -> code 1
  Trial 11 (t=1594.423s): 0.0000000000e+00 -> code 0
  Trial 12 (t=1758.157s): 0.0000000000e+00 -> code 0

Classical music run type: task-classicalMusic
Expected behavior: All trials should reference the sam

In [ ]:
# Test the updated BCMIFMRILoader implementation
print("🚀 TESTING UPDATED BCMIFMRILoader")
print("=" * 50)

# Reload the module to get the updated code
import importlib
import bcmi
importlib.reload(bcmi)

# Test genMusic01 with updated implementation
print("Testing genMusic01:")
try:
    gen_loader = bcmi.BCMIFMRILoader('./datasets/bcmi/bcmi-fmri')
    gen_data = gen_loader.load_subject_data('01', max_runs=1)
    
    # Check if we have any runs loaded
    if '01' in gen_data and gen_data['01']:
        print(f"  ✅ Loaded successfully")
        
        # Look for genMusic runs
        for run_name, run_data in gen_data['01'].items():
            if 'genMusic' in run_name:
                print(f"  Processing {run_name}:")
                print(f"    Recording duration: {run_data['raw'].times[-1]:.1f}s")
                
                # Check fMRI trials
                if 'fmri_trials' in run_data:
                    trials = run_data['fmri_trials']
                    print(f"    Number of trials: {len(trials)}")
                    
                    for trial in trials[:3]:  # Show first 3 trials
                        print(f"      Trial {trial['trial_number']}: onset={trial['onset']:.1f}s, "
                              f"music_code={trial['music_code']}, file={trial['music_file']}")
                else:
                    print("    ❌ No fMRI trials found")
                break
    else:
        print("  ❌ Failed to load data")
        
except Exception as e:
    print(f"  ❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\nTesting classicalMusic:")
try:
    # Load just one run to test classicalMusic
    all_data = gen_loader.load_all_subjects(max_subjects=1, max_runs_per_subject=10)
    
    if '01' in all_data and all_data['01']:
        # Look for classicalMusic run
        for run_name, run_data in all_data['01'].items():
            if 'classicalMusic' in run_name:
                print(f"  ✅ Found {run_name}")
                print(f"    Recording duration: {run_data['raw'].times[-1]:.1f}s")
                
                # Check fMRI trials
                if 'fmri_trials' in run_data:
                    trials = run_data['fmri_trials']
                    print(f"    Number of trials: {len(trials)}")
                    
                    for trial in trials[:3]:  # Show first 3 trials
                        print(f"      Trial {trial['trial_number']}: onset={trial['onset']:.1f}s, "
                              f"music_code={trial['music_code']}, file={trial['music_file']}")
                else:
                    print("    ❌ No fMRI trials found")
                break
        else:
            print("  ❌ No classicalMusic run found")
    else:
        print("  ❌ Failed to load data")
        
except Exception as e:
    print(f"  ❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\n✅ Updated implementation test completed!")

🚀 TESTING UPDATED BCMIFMRILoader


ModuleNotFoundError: No module named 'bcmi'

In [ ]:
import openneuro as on

# Download entire dataset
on.download(dataset='ds003774', target_dir='./musin_g_data')


👋 Hello! This is openneuro-py 2025.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds003774 …


📁 Traversing directories for ds003774 : 0 entities [00:00, ? entities/s]

📥 Retrieving up to 1840 files (5 concurrent downloads). 
✅ Finished downloading ds003774.
 
🧠 Please enjoy your brains.
 


🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub-005/ses-08/eeg/sub-005_ses-08_task-MusicListening_run-8_eeg.set, retrying in 0.5 sec …
🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub-005/ses-08/eeg/sub-005_ses-08_task-MusicListening_run-8_electrodes.tsv, retrying in 0.5 sec …
🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub-005/ses-08/eeg/sub-005_ses-08_task-MusicListening_run-8_events.json, retrying in 0.5 sec …
🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub-005/ses-09/eeg/sub-005_ses-09_task-MusicListening_run-9_channels.tsv, retrying in 0.5 sec …
🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub-005/ses-08/eeg/sub-005_ses-08_task-MusicListening_run-8_events.json, retrying in 0.5 sec …
🔄 Request timed out while downloading /home/zmrocze/studia/uwr/magisterka/musin_g_data/sub

In [ ]:
%load_ext autoreload
%autoreload 2

from nmed_t import NMEDTLoader, main

main("./datasets/nmed-t")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=== NMED-T Dataset Summary ===
Dataset: Naturalistic Music EEG Dataset - Tempo (NMED-T)
Participants: 20
Songs: 10
Electrodes (cleaned): 125
Sampling rate (cleaned): 125 Hz

=== Loading Cleaned EEG Data ===
Loaded song 1: First Fires - Shape: (125, 34795, 20)
Loaded song 1: First Fires - Shape: (125, 34795, 20)
Loaded song 2: Oino - Shape: (125, 33891, 20)
Loaded song 2: Oino - Shape: (125, 33891, 20)
Loaded song 3: Tiptoes - Shape: (125, 34469, 20)
Song 1: First Fires by Bonobo
  Tempo: 55.97 BPM
  EEG shape: (125, 34795, 20) - (electrodes=125, time_samples, participants=20)
  Sampling rate: 125 Hz
Song 2: Oino by LA Priest
  Tempo: 69.44 BPM
  EEG shape: (125, 33891, 20) - (electrodes=125, time_samples, participants=20)
  Sampling rate: 125 Hz
Song 3: Tiptoes by Daedelus
  Tempo: 74.26 BPM
  EEG shape: (125, 34469, 20) - (electrodes=125, time_samples, participants=20)
  Sampling rate: 125 Hz

=== 

In [2]:
# Comprehensive MUSIN-G Dataset Loader Based on Research
# Dataset described in: Miyapuram et al. (2022), Data in Brief, doi:10.1016/j.dib.2022.108663

import mne
import mne_bids
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🎵 MUSIN-G Dataset: Complete Import and Description")
print("=" * 70)
print("Based on: Miyapuram et al. (2022) - Electroencephalography (EEG) dataset")
print("during naturalistic music listening comprising different genres")
print("🔗 https://openneuro.org/datasets/ds003774")
print()

# Dataset paths
# EXPERIMENTAL PROTOCOL (from research)
print("🧪 EXPERIMENTAL PROTOCOL:")
print("• Participants: 20 Indian participants (16 male, 4 female, avg age 25.3±3.38)")
print("• EEG System: 128-channel Hydrocel Geodesic Sensor Net (HCGSN)")
print("• Recording: NetStation 5.4, referenced to Cz, 250Hz sampling")
print("• Location: Indian Institute of Technology Gandhinagar, India")
print()

print("📋 EXPERIMENTAL PROCEDURE:")
print("1. Single beep → Participant closes eyes")
print("2. Song plays (~2 minutes) → EEG recorded with eyes closed")
print("3. Double beep → Participant opens eyes")
print("4. Ratings collected: Familiarity & Enjoyment (1-5 scale)")
print("   - 1 = Most familiar/enjoyable, 5 = Least familiar/enjoyable")
print("5. Repeat for all 12 songs")
print()

bids_root = '/home/zmrocze/studia/uwr/magisterka/datasets/musin_g_data'
# Load behavioral data

from musing import MUSINGDataset

# Initialize and demonstrate
print("🚀 Initializing Complete MUSIN-G Dataset Loader...")
complete_dataset = MUSINGDataset(bids_root)


behavioral_data = complete_dataset.behavioral_data
print("📊 BEHAVIORAL DATA SUMMARY:")
print(f"• Total ratings: {len(behavioral_data)} (20 subjects × 12 songs)")
print(f"• Enjoyment range: {behavioral_data['Enjoyment'].min()}-{behavioral_data['Enjoyment'].max()}")
print(f"• Familiarity range: {behavioral_data['Familiarity'].min()}-{behavioral_data['Familiarity'].max()}")
print(f"• Mean enjoyment: {behavioral_data['Enjoyment'].mean():.2f}±{behavioral_data['Enjoyment'].std():.2f}")
print(f"• Mean familiarity: {behavioral_data['Familiarity'].mean():.2f}±{behavioral_data['Familiarity'].std():.2f}")
print()

print(f"\n🎯 Dataset ready for complete loading!")
print(f"• Use: complete_dataset.load_complete_dataset(max_subjects=5) for demo")
print(f"• Use: complete_dataset.load_complete_dataset() for full dataset")
musin_data = complete_dataset.load_complete_dataset()
complete_dataset.get_dataset_statistics()
print(f"• Use: complete_dataset.get_dataset_statistics() for analysis")

🎵 MUSIN-G Dataset: Complete Import and Description
Based on: Miyapuram et al. (2022) - Electroencephalography (EEG) dataset
during naturalistic music listening comprising different genres
🔗 https://openneuro.org/datasets/ds003774

🧪 EXPERIMENTAL PROTOCOL:
• Participants: 20 Indian participants (16 male, 4 female, avg age 25.3±3.38)
• EEG System: 128-channel Hydrocel Geodesic Sensor Net (HCGSN)
• Recording: NetStation 5.4, referenced to Cz, 250Hz sampling
• Location: Indian Institute of Technology Gandhinagar, India

📋 EXPERIMENTAL PROCEDURE:
1. Single beep → Participant closes eyes
2. Song plays (~2 minutes) → EEG recorded with eyes closed
3. Double beep → Participant opens eyes
4. Ratings collected: Familiarity & Enjoyment (1-5 scale)
   - 1 = Most familiar/enjoyable, 5 = Least familiar/enjoyable
5. Repeat for all 12 songs

🚀 Initializing Complete MUSIN-G Dataset Loader...
📊 BEHAVIORAL DATA SUMMARY:
• Total ratings: 240 (20 subjects × 12 songs)
• Enjoyment range: nan-nan
• Familiarity

In [ ]:
# Iterate over all subjects and sessions in complete_dataset
for subject_id, subject_data in complete_dataset.all_data.items():
  # print(f"Subject: {subject_id}")
  for session_id, session_data in subject_data.items():
    song_name = session_data['song_info']['name']
    enjoyment = session_data['enjoyment']
    familiarity = session_data['familiarity']
    # print(f"  Session: {session_id} | Song: {song_name} | Enjoyment: {enjoyment} | Familiarity: {familiarity}")

In [3]:
# Demonstration: Load subset and perform analyses
print("🔍 DEMONSTRATION: Loading subset of MUSIN-G dataset")
print("=" * 60)

# Load first 3 subjects as demonstration (to avoid overwhelming output)
demo_data = complete_dataset.load_complete_dataset(max_subjects=3, verbose=True)

# Get dataset statistics
complete_dataset.get_dataset_statistics()

print("\n" + "="*60)
print("📊 EXAMPLE ANALYSES")
print("="*60)

# Example 1: Genre preference analysis
print("\n1️⃣ GENRE PREFERENCE ANALYSIS (Demo subjects):")
genre_enjoyment = {}
genre_familiarity = {}

for subject in demo_data:
    for session in demo_data[subject]:
        data_point = demo_data[subject][session]
        genre = data_point['song_info']['genre']
        enjoyment = data_point['enjoyment']
        familiarity = data_point['familiarity']
        
        if genre not in genre_enjoyment:
            genre_enjoyment[genre] = []
            genre_familiarity[genre] = []
        
        if enjoyment is not None:
            genre_enjoyment[genre].append(enjoyment)
        if familiarity is not None:
            genre_familiarity[genre].append(familiarity)

for genre in genre_enjoyment:
    if genre_enjoyment[genre]:
        mean_enjoy = np.mean(genre_enjoyment[genre])
        mean_familiar = np.mean(genre_familiarity[genre])
        print(f"  {genre:25} | Enjoy: {mean_enjoy:.1f}/5 | Familiar: {mean_familiar:.1f}/5")

# Example 2: EEG data quality check
print(f"\n2️⃣ EEG DATA QUALITY CHECK:")
for subject in list(demo_data.keys())[:2]:  # Check first 2 subjects
    print(f"\nSubject {subject}:")
    for session in ['01', '07', '12']:  # Sample: Electronic, Classical, Rock
        if session in demo_data[subject]:
            raw = demo_data[subject][session]['raw']
            song_name = demo_data[subject][session]['song_info']['name']
            
            # Basic quality metrics
            duration = raw.times[-1]
            n_channels = raw.info['nchan']
            sfreq = raw.info['sfreq']
            
            # Check for bad channels (simple amplitude check)
            data = raw.get_data()
            channel_std = np.std(data, axis=1)
            outlier_channels = np.sum(channel_std > 3 * np.median(channel_std))
            
            print(f"  Song {session} ({song_name[:20]}...)")
            print(f"    Duration: {duration:.1f}s | Channels: {n_channels} | Freq: {sfreq}Hz")
            print(f"    Potential bad channels: {outlier_channels}")

# Example 3: Cross-cultural music analysis setup
print(f"\n3️⃣ CROSS-CULTURAL MUSIC CATEGORIES:")
categories = {
    'Western_Electronic': ['Deep House', 'Electronics', 'Electronic Dance', 'Ambient'],
    'Western_Traditional': ['Indie', 'New Age', 'Soft Jazz', 'Goth Rock', 'Progressive Instrumental Rock'],
    'Indian_Traditional': ['Hindustani Classical', 'Indian Semi-Classical', 'Indian Folk']
}

for category, genres in categories.items():
    matching_songs = [s for s in songs_info_enhanced.values() if s['genre'] in genres]
    print(f"  {category}: {len(matching_songs)} songs")
    for song in matching_songs:
        print(f"    - {song['name']} ({song['genre']})")

print(f"\n🎯 READY FOR ADVANCED ANALYSES:")
print(f"• Genre classification from EEG")
print(f"• Cross-cultural music perception")
print(f"• Individual vs. group music preferences")
print(f"• Temporal dynamics of music listening")
print(f"• Familiarity vs. enjoyment neural correlates")

print(f"\n💡 NEXT STEPS:")
print(f"# Load complete dataset:")
print(f"# all_data = complete_dataset.load_complete_dataset()")
print(f"# ")
print(f"# Access specific data:")
print(f"# subject_001_data = complete_dataset.get_subject_data('001')")
print(f"# classical_song_data = complete_dataset.get_song_data_across_subjects('07')")
print(f"# ")
print(f"# Access song metadata from Raw object:")
print(f"# raw = subject_001_data['01']['raw']")
print(f"# metadata = complete_dataset.get_song_metadata(raw)")
print(f"# print(metadata['genre'], metadata['enjoyment_rating'])")

🔍 DEMONSTRATION: Loading subset of MUSIN-G dataset
🔄 Loading complete MUSIN-G dataset...
📦 Subjects to load: 3 of 20

Subject 001 (1/3):
  ✓ Song 01: Trip to the lonely planet (Deep House) | Enjoy: None/5, Familiar: None/5
  ✓ Song 02: Sail (Indie) | Enjoy: None/5, Familiar: None/5
  ✓ Song 03: Concept 15 (Electronics) | Enjoy: None/5, Familiar: None/5
  ✓ Song 04: Aurore (New Age) | Enjoy: None/5, Familiar: None/5
  ✓ Song 05: Proof (Electronic Dance) | Enjoy: None/5, Familiar: None/5
  ✓ Song 06: Glider (Ambient) | Enjoy: None/5, Familiar: None/5
  ✓ Song 07: Raag Bihag (Hindustani Classical) | Enjoy: None/5, Familiar: None/5
  ✓ Song 08: Albela sajan (Indian Semi-Classical) | Enjoy: None/5, Familiar: None/5
  ✓ Song 09: Mor Bani Thanghat Kare (Indian Folk) | Enjoy: None/5, Familiar: None/5
  ✓ Song 10: Fallin (Soft Jazz) | Enjoy: None/5, Familiar: None/5
  ✓ Song 11: Master of Running (Goth Rock) | Enjoy: None/5, Familiar: None/5
  ✓ Song 12: JB (Progressive Instrumental Rock) | Enj

NameError: name 'songs_info_enhanced' is not defined

In [ ]:
from openmiir import main as openmiir_main
openmiir_main()

🎵 OpenMIIR Dataset Loader
Available subjects: ['01', '02', '03', '04', '05']... (showing first 5)

📦 Loading first 3 subjects (2 runs each)...
🔄 Loading OpenMIIR dataset...
📦 Subjects to load: 3 of 19
🏃 Runs per subject: 2

Loading subject 01:
  ✓ Run 1: 859.0s, 36 events, 18 trials
  ✓ Run 2: 841.0s, 36 events, 18 trials
Loading subject 02:
  ✓ Run 1: 1025.0s, 36 events, 18 trials
  ✓ Run 2: 979.0s, 36 events, 18 trials
Loading subject 03:
  ✓ Run 1: 865.0s, 36 events, 18 trials
  ✓ Run 2: 828.0s, 36 events, 18 trials

📊 LOADING SUMMARY:
• Successfully loaded: 3 subjects
• Failed to load: 0 subjects
• Success rate: 100.0%
📈 DATASET STATISTICS:
• Loaded subjects: 3
• Total runs loaded: 6
• Total trials: 108

🎭 EMOTIONAL CONDITION DISTRIBUTION:
  Condition 1:   6 trials
  Condition 2:  18 trials
  Condition 3:  10 trials
  Condition 4:  20 trials
  Condition 5:  18 trials
  Condition 6:  12 trials
  Condition 7:  12 trials
  Condition 9:  12 trials

⚙️  TECHNICAL SPECIFICATIONS:
• Sampl

/home/zmrocze/studia/uwr/magisterka/src/openmiir.py:53: RuntimeWarning: Invalid measurement date encountered in the header.
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/openmiir.py:53: RuntimeWarning: No BIDS -> MNE mapping found for channel type "UserReport". Type of channel "VA1" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/openmiir.py:53: RuntimeWarning: No BIDS -> MNE mapping found for channel type "UserReport". Type of channel "VA2" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/openmiir.py:53: RuntimeWarning: No BIDS -> MNE mapping found for channel type "MetaData". Type of channel "VAtarg" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/openmiir.py:53: RuntimeWarning: Invalid measurement date encountered in the header.
  raw = read_raw_bids(bids_p

# 🎵 Music EEG Datasets Summary

## Successfully Imported Datasets

### 1. MUSIN-G Dataset (Naturalistic Music Listening)
- **Source**: Miyapuram et al. (2022), OpenNeuro ds003774
- **Participants**: 20 Indian participants  
- **EEG**: 128-channel Hydrocel Geodesic Sensor Net, 250Hz
- **Music**: 12 real songs across genres (Indian classical, Western electronic, etc.)
- **Design**: Naturalistic listening with eyes closed, ~2 minutes per song
- **Behavioral**: Familiarity & enjoyment ratings (1-5 scale)
- **Use cases**: Genre classification, cross-cultural music perception, preference analysis

### 2. OpenMIIR Dataset (Affective Music BCI)
- **Source**: Daly et al. (2018), OpenNeuro ds002722  
- **Participants**: 19 healthy adults from University of Reading
- **EEG**: 32-channel system + ECG + GSR, 1000Hz sampling
- **Music**: Synthetic music clips (40 seconds each)
- **Design**: 3×3 factorial (valence × arousal) = 9 emotional conditions
- **Structure**: 5 runs × 18 trials = 90 trials per participant
- **Use cases**: Emotion classification, valence/arousal analysis, affective BCI

## Key Differences

| Aspect | MUSIN-G | OpenMIIR |
|--------|---------|----------|
| Music Type | Real songs | Synthetic clips |
| Duration | ~2 minutes | 40 seconds |
| Focus | Genre/Cultural | Emotional states |
| EEG Channels | 128 | 32 (+ECG/GSR) |
| Sampling Rate | 250 Hz | 1000 Hz |
| Behavioral | Ratings | Valence/Arousal |

Both datasets are now successfully loaded and ready for analysis!

In [5]:
from bcmi import create_bcmi_loader
loader = create_bcmi_loader('./datasets/bcmi/bcmi-training')
data = loader.load_all_subjects()
loader.get_dataset_statistics()

# Load specific subjects and runs
subject_data = loader.load_subject_data('01', max_runs=3)

exp_info = loader._get_experimental_info()
print(exp_info)

# Get trials for specific emotional states
happy_trials = loader.get_condition_trials(1)  # High valence, high arousal
calm_trials = loader.get_condition_trials(8)   # Neutral valence, low arousal

subjects = loader.get_available_subjects()

# Access raw EEG data and events
raw_eeg = loader.data[subjects[0]]['3']
events = loader.data[subjects[0]]['3']['1']['processed_events']

# Load all BCMI datasets at once
# from bcmi import load_all_bcmi_datasets
# all_loaders = load_all_bcmi_datasets('./datasets/bcmi', max_subjects_per_dataset=3)

# # Compare across datasets
# for name, loader in all_loaders.items():
#     print(f"{name}: {len(loader.data)} subjects")
#     loader.get_dataset_statistics()

Loading subject 08 (bcmi-training):
  ✓ Run 1 ses-3: 1008.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1005.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1010.0s, 36 events, 36 trials
  ✓ Run 4 ses-3: 1011.0s, 36 events, 36 trials
Loading subject 09 (bcmi-training):
  ✓ Run 1 ses-3: 1038.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1052.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1034.0s, 36 events, 36 trials
  ✓ Run 4 ses-3: 1036.0s, 36 events, 36 trials
  ✓ Run 1 ses-1: 1045.0s, 36 events, 36 trials
  ✓ Run 2 ses-1: 1033.0s, 36 events, 36 trials
  ✓ Run 3 ses-1: 1034.0s, 36 events, 36 trials
  ✓ Run 4 ses-1: 1031.0s, 36 events, 36 trials
  ✓ Run 1 ses-2: 1036.0s, 36 events, 36 trials
  ✓ Run 2 ses-2: 1032.0s, 36 events, 36 trials
  ✓ Run 3 ses-2: 1016.0s, 36 events, 36 trials
  ✓ Run 4 ses-2: 1026.0s, 36 events, 36 trials
Loading subject 10 (bcmi-training):
  ✓ Run 1 ses-3: 1095.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1115.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1151.0s, 36 events, 36 trials

In [19]:
loader = create_bcmi_loader('./datasets/bcmi/bcmi-training')
data = loader.load_all_subjects()

Loading subject 08 (bcmi-training):
  ✓ Run 1 ses-3: 1008.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1005.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1010.0s, 36 events, 36 trials
  ✓ Run 4 ses-3: 1011.0s, 36 events, 36 trials
Loading subject 09 (bcmi-training):
  ✓ Run 1 ses-3: 1038.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1052.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1034.0s, 36 events, 36 trials
  ✓ Run 4 ses-3: 1036.0s, 36 events, 36 trials
  ✓ Run 1 ses-1: 1045.0s, 36 events, 36 trials
  ✓ Run 2 ses-1: 1033.0s, 36 events, 36 trials
  ✓ Run 3 ses-1: 1034.0s, 36 events, 36 trials
  ✓ Run 4 ses-1: 1031.0s, 36 events, 36 trials
  ✓ Run 1 ses-2: 1036.0s, 36 events, 36 trials
  ✓ Run 2 ses-2: 1032.0s, 36 events, 36 trials
  ✓ Run 3 ses-2: 1016.0s, 36 events, 36 trials
  ✓ Run 4 ses-2: 1026.0s, 36 events, 36 trials
Loading subject 10 (bcmi-training):
  ✓ Run 1 ses-3: 1095.0s, 36 events, 36 trials
  ✓ Run 2 ses-3: 1115.0s, 36 events, 36 trials
  ✓ Run 3 ses-3: 1151.0s, 36 events, 36 trials

In [7]:
import numpy as np
import mne
from bcmi import create_bcmi_loader
# raw1 = loader.data[subjects[0]]['3']['1']['raw']
# events = loader.data[subjects[0]]['3']['1']['events']
# loader = create_bcmi_loader('./datasets/bcmi/bcmi-training')
loader = create_bcmi_loader('./datasets/bcmi/bcmi-calibration')
data = loader.load_all_subjects()
loader.get_dataset_statistics()

Loading subject 01 (bcmi-calibration):
  ✓ Run 1: 859.0s, 36 events, 18 trials
  ✓ Run 2: 841.0s, 36 events, 18 trials
  ✓ Run 3: 844.0s, 36 events, 18 trials
  ✓ Run 4: 832.0s, 36 events, 18 trials
  ✓ Run 5: 815.0s, 36 events, 18 trials
Loading subject 02 (bcmi-calibration):
  ✓ Run 1: 1025.0s, 36 events, 18 trials
  ✓ Run 2: 979.0s, 36 events, 18 trials
  ✓ Run 3: 963.0s, 36 events, 18 trials
  ✓ Run 4: 919.0s, 36 events, 18 trials
  ✓ Run 5: 886.0s, 36 events, 18 trials
Loading subject 03 (bcmi-calibration):
  ✓ Run 1: 865.0s, 36 events, 18 trials
  ✓ Run 2: 828.0s, 36 events, 18 trials
  ✓ Run 3: 820.0s, 36 events, 18 trials
  ✓ Run 4: 811.0s, 36 events, 18 trials
  ✓ Run 5: 817.0s, 36 events, 18 trials
Loading subject 04 (bcmi-calibration):
  ✓ Run 1: 984.0s, 36 events, 18 trials
  ✓ Run 2: 935.0s, 36 events, 18 trials
  ✓ Run 3: 868.0s, 36 events, 18 trials
  ✓ Run 4: 831.0s, 36 events, 18 trials
  ✓ Run 5: 800.0s, 36 events, 18 trials
Loading subject 05 (bcmi-calibration):
  ✓ 

In [ ]:
# durations
# epochs = mne.Epochs(
#     raw1,
#     events,
#     event_id={'stimulus_onset': 1},
#     tmin=-0.2,      # Pre-stimulus baseline
#     tmax=0.8,       # Post-stimulus window  
#     baseline=(-0.2, 0),
#     preload=True
# )
# session = '3'
# for x in loader.data[subjects[0]][session]['1']['processed_events']['training_pairs']:
#   i,j = x['affect_1']['code'], x['affect_2']['code']
#   assert  x['affect_2']['onset'] - x['affect_1']['onset'] == 20_000

def onset_secs_to_samples(onset_secs, sfreq):
  return round(onset_secs * sfreq)

def loader_data_iter(data):
  for subject in data.keys():
    for session in data[subject].keys():
      for run in data[subject][session].keys():
        r = data[subject][session][run]
        yield subject, session, run, r

for subject, session, run, r in loader_data_iter(loader.data):
  for x in r['processed_events']['training_pairs']:
    i,j = x['affect_1']['code'], x['affect_2']['code']
    # music: i-j_session
    # print(x['affect_2']['onset'] - x['affect_1']['onset'])
    assert onset_secs_to_samples(x['affect_2']['onset'] - x['affect_1']['onset'], sfreq=r['raw'].info['sfreq']) == 20_000


In [20]:
loader.data["08"]['3']['1']

{'raw': <RawEDF | sub-08_ses-3_task-run1_eeg.edf, 37 x 1008000 (1008.0 s), ~33 KiB, data not loaded>,
 'events':       onset  duration  trial_type
 0     8.865        21           1
 1    28.865        21           6
 2    66.732        21           9
 3    86.732        21           6
 4   122.835        21           9
 5   142.835        21           3
 6   177.397        21           3
 7   197.397        21           6
 8   232.831        21           3
 9   252.831        21           4
 10  288.196        21           9
 11  308.196        21           8
 12  343.341        21           1
 13  363.341        21           2
 14  397.975        21           1
 15  417.975        21           8
 16  455.042        21           8
 17  475.042        21           7
 18  510.600        21           1
 19  530.600        21           9
 20  567.530        21           6
 21  587.530        21           8
 22  622.195        21           1
 23  642.195        21           3
 24  677.602 

In [ ]:

wav_filenames_ordered = [
  'hvha1.wav', 'hvha10.wav', 'hvha11.wav', 'hvha12.wav', 'hvha2.wav', 'hvha3.wav', 'hvha4.wav', 
  'hvha5.wav', 'hvha6.wav', 'hvha7.wav', 'hvha8.wav', 'hvha9.wav', 'hvla1.wav', 'hvla10.wav', 
  'hvla11.wav', 'hvla12.wav', 'hvla2.wav', 'hvla3.wav', 'hvla4.wav', 'hvla5.wav', 'hvla6.wav',
  'hvla7.wav', 'hvla8.wav', 'hvla9.wav', 'hvna1.wav', 'hvna10.wav', 'hvna11.wav', 'hvna12.wav',
  'hvna2.wav', 'hvna3.wav', 'hvna4.wav', 'hvna5.wav', 'hvna6.wav', 'hvna7.wav' , 'hvna8.wav',
  'hvna9.wav', 'lvha1.wav', 'lvha10.wav', 'lvha11.wav', 'lvha12.wav', 'lvha2.wav', 'lvha3.wav',
  'lvha4.wav', 'lvha5.wav','lvha6.wav', 'lvha7.wav', 'lvha8.wav', 'lvha9.wav', 'lvla1.wav',
  'lvla10.wav', 'lvla11.wav', 'lvla12.wav', 'lvla2.wav', 'lvla3.wav', 'lvla4.wav', 'lvla5.wav',
  'lvla6.wav', 'lvla7.wav', 'lvla8.wav' , 'lvla9.wav', 'lvna1.wav', 'lvna10.wav', 'lvna11.wav',
  'lvna12.wav', 'lvna2.wav', 'lvna3.wav', 'lvna4.wav', 'lvna5.wav', 'lvna6.wav', 'lvna7.wav',
  'lvna8.wav', 'lvna9.wav', 'nvha1.wav', 'nvha10.wav', 'nvha11.wav', 'nvha12.wav', 'nvha2.wav',
  'nvha3.wav', 'nvha4.wav', 'nvha5.wav', 'nvha6.wav', 'nvha7.wav', 'nvha8.wav', 'nvha9.wav',
  'nvla1.wav', 'nvla10.wav', 'nvla11.wav', 'nvla12.wav', 'nvla2.wav', 'nvla3.wav', 'nvla4.wav',
  'nvla5.wav', 'nvla6.wav', 'nvla7.wav', 'nvla8.wav', 'nvla9.wav', 'nvna1.wav', 'nvna10.wav',
  'nvna11.wav', 'nvna12.wav', 'nvna2.wav', 'nvna3.wav', 'nvna4.wav', 'nvna5.wav', 'nvna6.wav', 
  'nvna7.wav', 'nvna8.wav', 'nvna9.wav'
]

duration = 21000
for subject, session, run, r in loader.loader_data_iter():
  # print(r['processed_events']['marker_events'])
  for _, marker in r['processed_events']['marker_events'].iterrows():
    # print(r['processed_events']['marker_events'])
    print(marker)
    onset = marker['onset']
    music_file = wav_filenames_ordered[ round(marker['trial_type']) - 100 ]
    print(f"Onset (samples): {onset}, Music file: {music_file}, emotion: {marker}")

onset           9.643
duration       21.000
trial_type    104.000
Name: 1, dtype: float64
Onset (samples): 9.643, Music file: hvha2.wav, emotion: onset           9.643
duration       21.000
trial_type    104.000
Name: 1, dtype: float64
onset          63.82
duration       21.00
trial_type    163.00
Name: 3, dtype: float64
Onset (samples): 63.82, Music file: lvna12.wav, emotion: onset          63.82
duration       21.00
trial_type    163.00
Name: 3, dtype: float64
onset         115.429
duration       21.000
trial_type    173.000
Name: 5, dtype: float64
Onset (samples): 115.429, Music file: nvha10.wav, emotion: onset         115.429
duration       21.000
trial_type    173.000
Name: 5, dtype: float64
onset         163.065
duration       21.000
trial_type    205.000
Name: 7, dtype: float64
Onset (samples): 163.065, Music file: nvna7.wav, emotion: onset         163.065
duration       21.000
trial_type    205.000
Name: 7, dtype: float64
onset         209.917
duration       21.000
trial_type  

In [ ]:
loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')
loader.load_all_subjects()

NameError: name 'create_bcmi_loader' is not defined

In [ ]:
# Test the updated BCMIFMRILoader
from bcmi import create_bcmi_loader
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 120)

# Load the fMRI dataset
fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')
fmri_loader.load_all_subjects(max_subjects=1)

# Access and display the processed trials for a generated music run
subject_id = fmri_loader.subjects[0]
gen_music_run = 'genMusic01'
classical_music_run = 'classicalMusic'

if subject_id in fmri_loader.data and gen_music_run in fmri_loader.data[subject_id]['no_session']:
    print(f"--- Processed Trials for {gen_music_run} ---")
    gen_trials = fmri_loader.data[subject_id]['no_session'][gen_music_run]['processed_events']['fmri_trials']
    display(pd.DataFrame(gen_trials))

if subject_id in fmri_loader.data and classical_music_run in fmri_loader.data[subject_id]['no_session']:
    print(f"\n--- Processed Trials for {classical_music_run} ---")
    classical_trials = fmri_loader.data[subject_id]['no_session'][classical_music_run]['processed_events']['fmri_trials']
    display(pd.DataFrame(classical_trials))


{'raw': <RawEDF | sub-01_task-genMusic01_eeg.edf, 47 x 742000 (742.0 s), ~41 KiB, data not loaded>,
 'events':         onset  duration  trial_type
 0       0.001     0.001           1
 1       0.041     0.001          47
 2       0.161     0.001          47
 3       1.172     0.001         265
 4       2.161     0.001          47
 5       4.161     0.001          47
 6       4.645     0.001         265
 7       6.161     0.001          47
 8       6.523     0.001         265
 9       8.161     0.001          47
 10      8.345     0.001         265
 11     10.161     0.001          47
 12     11.033     0.001         265
 13     12.161     0.001          47
 14     13.880     0.001         265
 15     14.161     0.001          47
 16     15.796     0.001         265
 17     16.161     0.001          47
 18     16.821     0.001         265
 19     18.161     0.001          47
 20     18.736     0.001         265
 21     20.161     0.001          47
 22     21.643     0.001         265
 2

In [ ]:
loader.data[loader.subjects[0]]['no_session']['classicalMusic']

{'raw': <RawEDF | sub-01_task-classicalMusic_eeg.edf, 47 x 1915000 (1915.0 s), ~41 KiB, data not loaded>,
 'events':          onset  duration  trial_type
 0        0.001     0.001           1
 1        0.001     0.001          47
 2        0.001     0.001        1283
 3        0.913     0.001        1285
 4        0.913     0.001         265
 5        1.501     0.001       34053
 6        1.801     0.001          47
 7        3.801     0.001          47
 8        3.836     0.001         265
 9        4.845     0.001         265
 10       5.801     0.001          47
 11       6.811     0.001         265
 12       7.801     0.001          47
 13       7.836     0.001         265
 14       9.801     0.001          47
 15      10.794     0.001         265
 16      11.801     0.001          47
 17      13.759     0.001         265
 18      13.801     0.001          47
 19      14.828     0.001         265
 20      15.801     0.001          47
 21      16.888     0.001         265
 22      1

In [ ]:
# Test the updated BCMIFMRILoader implementation
%load_ext autoreload
%autoreload 2

from bcmi import create_bcmi_loader
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 120)

print("🧪 Testing BCMI-fMRI Loader Implementation")
print("=" * 60)

# Load the fMRI dataset
fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')
fmri_loader.load_all_subjects(max_subjects=1)

# Access the first subject's data
subject_id = fmri_loader.subjects[0]
print(f"\n📊 Testing with subject: {subject_id}")

# Test genMusic trials
print(f"\n🎵 TESTING genMusic TRIALS:")
for gen_task in ['genMusic01', 'genMusic02', 'genMusic03']:
    if gen_task in fmri_loader.data[subject_id]['no_session']:
        print(f"\n--- {gen_task} ---")
        task_data = fmri_loader.data[subject_id]['no_session'][gen_task]
        
        # Show raw events with 768 markers
        events_768 = task_data['events'][task_data['events']['trial_type'] == 768]
        print(f"Raw 768 events found: {len(events_768)}")
        if len(events_768) > 0:
            print("First few 768 events:")
            print(events_768.head())
        
        # Show processed trials
        fmri_trials = task_data['processed_events'].get('fmri_trials', [])
        print(f"Processed trials: {len(fmri_trials)}")
        if fmri_trials:
            for trial in fmri_trials[:3]:  # Show first 3 trials
                print(f"  Trial {trial['trial_number']}: onset={trial['onset']:.3f}s, "
                      f"music_file={trial['music_file']}, duration={trial['duration']}s")

# Test classicalMusic trials  
print(f"\n🎼 TESTING classicalMusic TRIALS:")
if 'classicalMusic' in fmri_loader.data[subject_id]['no_session']:
    classical_data = fmri_loader.data[subject_id]['no_session']['classicalMusic']
    
    # Show raw events with 768 markers
    events_768 = classical_data['events'][classical_data['events']['trial_type'] == 768]
    print(f"Raw 768 events found: {len(events_768)}")
    if len(events_768) > 0:
        print("First few 768 events:")
        print(events_768.head())
    
    # Show processed trials
    fmri_trials = classical_data['processed_events'].get('fmri_trials', [])
    print(f"Processed trials: {len(fmri_trials)}")
    if fmri_trials:
        for trial in fmri_trials:
            print(f"  Trial {trial['trial_number']}: onset={trial['onset']:.3f}s, "
                  f"music_file={trial['music_file']}, duration={trial['duration']:.1f}s")

print(f"\n✅ Test completed!")

In [ ]:
# Simple debug test for BCMI-fMRI loader
from bcmi import create_bcmi_loader
import pandas as pd

print("🔍 DEBUGGING BCMI-fMRI Loader")

# Load one subject
fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')

# Check what we have before processing
print(f"Available subjects: {fmri_loader.subjects[:3]}")

# Load just one subject
if fmri_loader.subjects:
    subject_id = fmri_loader.subjects[0]
    print(f"\nLoading subject: {subject_id}")
    
    try:
        fmri_loader.load_subject_data(subject_id, max_runs=2)  # Load only 2 runs for testing
        
        print(f"\nLoaded data structure:")
        for session_key, session_data in fmri_loader.data[subject_id].items():
            print(f"  Session: {session_key}")
            for run_key, run_data in session_data.items():
                print(f"    Run: {run_key}")
                if 'processed_events' in run_data:
                    fmri_trials = run_data['processed_events'].get('fmri_trials', [])
                    print(f"      fMRI trials: {len(fmri_trials)}")
                    if fmri_trials:
                        print(f"      First trial: {fmri_trials[0]}")
    except Exception as e:
        print(f"Error during loading: {e}")
        import traceback
        traceback.print_exc()

🔍 DEBUGGING BCMI-fMRI Loader
Available subjects: ['01', '02', '03']

Loading subject: 01
Loading subject 01 (bcmi-fmri):


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)


  ✓ Run classicalMusic: 1915.0s, 2256 events, 1 trials
  ✓ Run genMusic01: 742.0s, 1162 events, 1 trials

Loaded data structure:
  Session: no_session
    Run: classicalMusic
      fMRI trials: 1
      First trial: {'trial_number': 1, 'onset': np.float64(31.771), 'duration': np.float64(1883.228), 'music_file': 'classical/unknown_piece_for_classicalMusic.mp3', 'task_type': 'classicalMusic'}
    Run: genMusic01
      fMRI trials: 0


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_y" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channe

In [ ]:
# Test the CORRECTED BCMIFMRILoader implementation
%load_ext autoreload
%autoreload 2

from bcmi import create_bcmi_loader
import pandas as pd

print("🔧 Testing CORRECTED BCMI-fMRI Loader")
print("=" * 50)

# Load the fMRI dataset
fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')

# Load one subject with both genMusic and classicalMusic
subject_id = fmri_loader.subjects[0]
print(f"\nTesting with subject: {subject_id}")

try:
    fmri_loader.load_subject_data(subject_id, max_runs=3)  # Load classicalMusic + 2 genMusic runs
    
    print(f"\n✅ RESULTS:")
    for session_key, session_data in fmri_loader.data[subject_id].items():
        print(f"\nSession: {session_key}")
        for run_key, run_data in session_data.items():
            fmri_trials = run_data['processed_events'].get('fmri_trials', [])
            print(f"  {run_key}: {len(fmri_trials)} trials")
            
            # Show details for each trial
            for trial in fmri_trials:
                onset = trial['onset']
                duration = trial['duration']
                music_file = trial['music_file']
                task_type = trial['task_type']
                music_code = trial.get('music_code', 'N/A')
                
                print(f"    Trial {trial['trial_number']}: {onset:.3f}s -> {onset+duration:.1f}s")
                print(f"      File: {music_file}")
                print(f"      Type: {task_type}")
                if music_code != 'N/A':
                    print(f"      Music code: {music_code}")
                    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

🔧 Testing CORRECTED BCMI-fMRI Loader

Testing with subject: 01
Loading subject 01 (bcmi-fmri):
  ✓ Run classicalMusic: 1915.0s, 2256 events, 1 trials
  ✓ Run genMusic01: 742.0s, 1162 events, 1 trials
  ✓ Run genMusic02: 621.0s, 637 events, 1 trials

✅ RESULTS:

Session: no_session
  classicalMusic: 1 trials
    Trial 1: 31.771s -> 1915.0s
      File: classical/unknown_piece_for_classicalMusic.mp3
      Type: classicalMusic
  genMusic01: 0 trials
  genMusic02: 0 trials


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning:

In [ ]:
# Debug: Check what channels are actually in the EEG data
from bcmi import create_bcmi_loader

print("🔍 INSPECTING EEG CHANNELS")
print("=" * 40)

fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')
subject_id = fmri_loader.subjects[0]

# Load just one run to check channels
fmri_loader.load_subject_data(subject_id, max_runs=1)

if 'genMusic01' in fmri_loader.data[subject_id]['no_session']:
    raw = fmri_loader.data[subject_id]['no_session']['genMusic01']['raw']
    
    print(f"Channel names in genMusic01:")
    for i, ch_name in enumerate(raw.ch_names):
        print(f"  {i:2d}: {ch_name}")
    
    print(f"\nTotal channels: {len(raw.ch_names)}")
    
    # Check if 'music' channel exists
    if 'music' in raw.ch_names:
        print(f"\n✅ 'music' channel found at index: {raw.ch_names.index('music')}")
        
        # Sample some values from the music channel
        music_idx = raw.ch_names.index('music')
        music_data = raw.get_data(picks=[music_idx])[0]
        print(f"Music channel stats:")
        print(f"  Min: {music_data.min():.6f}")
        print(f"  Max: {music_data.max():.6f}")
        print(f"  Mean: {music_data.mean():.6f}")
        print(f"  First 10 values: {music_data[:10]}")
    else:
        print(f"\n❌ 'music' channel NOT found!")
        print("Available channels containing 'music' or similar:")
        for ch in raw.ch_names:
            if 'music' in ch.lower() or 'stimuli' in ch.lower():
                print(f"  - {ch}")
else:
    print("genMusic01 not loaded or available")

🔍 INSPECTING EEG CHANNELS
Loading subject 01 (bcmi-fmri):
  ✓ Run classicalMusic: 1915.0s, 2256 events, 1 trials
genMusic01 not loaded or available


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:298: RuntimeWarning:

In [ ]:
fmri_loader = create_bcmi_loader('./datasets/bcmi/bcmi-fmri')
fmri_loader.load_all_subjects()

🔄 Loading bcmi-fmri dataset...
📦 Subjects to load: 21 of 21

Loading subject 01 (bcmi-fmri):
  ✓ Run classicalMusic: 1915.0s, 2256 events, 1 trials
  ✓ Run genMusic01: 742.0s, 1162 events, 1 trials


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning:

  ✓ Run genMusic02: 621.0s, 637 events, 1 trials
  ✓ Run genMusic03: 578.0s, 595 events, 1 trials
  ✓ Run washout: 90.0s, 102 events, 1 trials
Loading subject 02 (bcmi-fmri):


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning:

  ✓ Run classicalMusic: 1898.0s, 1901 events, 1 trials
  ✓ Run genMusic01: 600.0s, 854 events, 1 trials
  ✓ Run genMusic02: 594.0s, 775 events, 1 trials
  ✓ Run genMusic03: 593.0s, 793 events, 1 trials
  ✓ Run washout: 112.0s, 125 events, 1 trials
Loading subject 03 (bcmi-fmri):


...

/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning:

  ✓ Run washout: 123.0s, 143 events, 1 trials
Loading subject 21 (bcmi-fmri):
  ✓ Run classicalMusic: 1891.0s, 1894 events, 1 trials
  ✓ Run genMusic01: 597.0s, 616 events, 1 trials


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning:

  ✓ Run genMusic02: 581.0s, 594 events, 1 trials
  ✓ Run genMusic03: 580.0s, 590 events, 1 trials
  ✓ Run washout: 103.0s, 130 events, 1 trials

📊 LOADING SUMMARY:
• Successfully loaded: 21 subjects
• Failed to load: 0 subjects
• Success rate: 100.0%


{'01': {'no_session': {'classicalMusic': {'raw': <RawEDF | sub-01_task-classicalMusic_eeg.edf, 47 x 1915000 (1915.0 s), ~41 KiB, data not loaded>,
    'events':          onset  duration  trial_type
    0        0.001     0.001           1
    1        0.001     0.001          47
    2        0.001     0.001        1283
    3        0.913     0.001        1285
    4        0.913     0.001         265
    ...        ...       ...         ...
    2251  1906.102     0.001         265
    2252  1907.801     0.001          47
    2253  1909.801     0.001          47
    2254  1911.801     0.001          47
    2255  1913.801     0.001          47
    
    [2256 rows x 3 columns],
    'processed_events': {'emotion_events':    onset  duration  trial_type emotion_description valence arousal
     0  0.001     0.001           1       Excited/Happy    High    High,
     'marker_events':          onset  duration  trial_type
     2        0.001     0.001        1283
     3        0.913     0.001    

In [ ]:
# Direct EEG data inspection
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd

print("🔍 DIRECT EEG DATA INSPECTION")
print("=" * 40)

# Load genMusic01 directly
bids_path = BIDSPath(
    subject='01',
    task='genMusic01',
    datatype='eeg',
    root='./datasets/bcmi/bcmi-fmri'
)

try:
    print("Loading raw EEG data...")
    raw = read_raw_bids(bids_path, verbose=False)
    
    print(f"\nChannel names ({len(raw.ch_names)} total):")
    for i, ch_name in enumerate(raw.ch_names):
        if 'music' in ch_name.lower() or 'stimuli' in ch_name.lower():
            print(f"  {i:2d}: {ch_name} ⭐")
        else:
            print(f"  {i:2d}: {ch_name}")
    
    # Check for music channel specifically
    if 'music' in raw.ch_names:
        print(f"\n✅ 'music' channel found!")
        
        # Load events to see trial onsets
        events_path = bids_path.copy().update(suffix='events', extension='.tsv')
        events_df = pd.read_csv(events_path.fpath, sep='\t')
        
        # Find 768 events (trial starts)
        trial_starts = events_df[events_df['trial_type'] == 768]
        print(f"\nTrial start events (768): {len(trial_starts)}")
        if len(trial_starts) > 0:
            print(trial_starts[['onset', 'duration', 'trial_type']].head())
            
            # Sample music channel at first trial start
            first_onset = trial_starts.iloc[0]['onset']
            music_idx = raw.ch_names.index('music')
            sample_idx = int(first_onset * raw.info['sfreq'])
            
            music_val = raw.get_data(picks=[music_idx])[0, sample_idx]
            music_code = int(music_val * 20)
            
            print(f"\nAt first trial (onset={first_onset:.3f}s):")
            print(f"  Music channel value: {music_val}")
            print(f"  Music code (val*20): {music_code}")
            print(f"  Formatted: {str(music_code).zfill(3)}")
            
            if len(str(music_code).zfill(3)) >= 3:
                s_code = str(music_code).zfill(3)
                music_file = f"{s_code[0]}-{s_code[1]}_{s_code[2]}.wav"
                print(f"  Music file: {music_file}")
    else:
        print(f"\n❌ 'music' channel not found!")
        
except Exception as e:
    print(f"❌ Error loading data: {e}")
    import traceback
    traceback.print_exc()

🔍 DIRECT EEG DATA INSPECTION
Loading raw EEG data...

Channel names (47 total):
   0: Fp1
   1: Fp2
   2: F3
   3: F4
   4: C3
   5: C4
   6: P3
   7: P4
   8: O1
   9: O2
  10: F7
  11: F8
  12: T7
  13: T8
  14: P7
  15: P8
  16: Fz
  17: Cz
  18: Pz
  19: Oz
  20: FC1
  21: FC2
  22: CP1
  23: CP2
  24: FC5
  25: FC6
  26: CP5
  27: CP6
  28: TP9
  29: TP10
  30: POz
  31: ECG
  32: ft_valance
  33: ft_arousal
  34: ft_x
  35: ft_y
  36: ft_ghostvalence
  37: ft_ghostarousal
  38: music ⭐
  39: trialtype
  40: sams_valence
  41: sams_arousal
  42: sams_valencert
  43: sams_arousalrt
  44: nback_stimuli ⭐
  45: nback_keypress
  46: Status

✅ 'music' channel found!

Trial start events (768): 24
       onset  duration  trial_type
242  191.151     0.001         768
243  191.158     0.001         768
333  238.056     0.001         768
334  238.060     0.001         768
406  283.447     0.001         768

At first trial (onset=191.151s):
  Music channel value: 1.4050086213473715e-05
  Mus

/tmp/ipykernel_26767/1809379087.py:18: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/1809379087.py:18: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/1809379087.py:18: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/1809379087.py:18: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/1809379087.py:18: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of chan

In [ ]:
# Sample music channel values at multiple trial onsets
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd
import numpy as np

print("🎵 SAMPLING MUSIC CHANNEL VALUES")
print("=" * 45)

# Load genMusic01 data
bids_path = BIDSPath(
    subject='01',
    task='genMusic01', 
    datatype='eeg',
    root='./datasets/bcmi/bcmi-fmri'
)

raw = read_raw_bids(bids_path, verbose=False)
events_path = bids_path.copy().update(suffix='events', extension='.tsv')
events_df = pd.read_csv(events_path.fpath, sep='\t')

# Find all 768 events (removing consecutive duplicates)
trial_starts = events_df[events_df['trial_type'] == 768].copy()
onsets = trial_starts['onset'].values
unique_onsets = onsets[np.concatenate([[True], np.diff(onsets) > 1])]

print(f"Total 768 events: {len(trial_starts)}")
print(f"Unique trial onsets (>1s apart): {len(unique_onsets)}")

# Sample music channel at each unique trial onset
music_idx = raw.ch_names.index('music')
sfreq = raw.info['sfreq']

print(f"\nMusic channel sampling at trial onsets:")
for i, onset in enumerate(unique_onsets):
    sample_idx = int(onset * sfreq)
    if sample_idx < raw.n_times:
        music_val = raw.get_data(picks=[music_idx])[0, sample_idx]
        music_code = int(music_val * 20) if music_val > 0 else 0
        
        # Format as 3-digit string
        s_code = str(music_code).zfill(3)
        if len(s_code) >= 3 and music_code > 0:
            music_file = f"generated/{s_code[0]}-{s_code[1]}_{s_code[2]}.wav"
        else:
            music_file = f"generated/silence_or_baseline.wav"
            
        print(f"  Trial {i+1:2d}: onset={onset:7.3f}s, val={music_val:12.6e}, "
              f"code={music_code:3d}, file={music_file}")

# Also check around the trial onsets to see if values change
print(f"\nChecking music channel around first trial onset:")
first_onset = unique_onsets[0]
start_sample = int((first_onset - 5) * sfreq)  # 5s before
end_sample = int((first_onset + 5) * sfreq)    # 5s after

if start_sample >= 0 and end_sample < raw.n_times:
    music_segment = raw.get_data(picks=[music_idx])[0, start_sample:end_sample]
    unique_vals = np.unique(music_segment)
    
    print(f"  Unique values in 10s window: {len(unique_vals)}")
    print(f"  Min: {music_segment.min():.6e}")
    print(f"  Max: {music_segment.max():.6e}")
    print(f"  Non-zero values: {len(music_segment[music_segment != 0])}")
    
    if len(unique_vals) > 1:
        print(f"  Sample unique values: {unique_vals[:10]}")

🎵 SAMPLING MUSIC CHANNEL VALUES


/tmp/ipykernel_26767/4019187128.py:17: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/4019187128.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/4019187128.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/4019187128.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/4019187128.py:17: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of chan

Total 768 events: 24
Unique trial onsets (>1s apart): 12

Music channel sampling at trial onsets:
  Trial  1: onset=191.151s, val=1.405009e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  2: onset=238.056s, val=2.259969e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  3: onset=283.447s, val=1.409969e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  4: onset=327.102s, val=4.105016e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  5: onset=373.110s, val=1.409969e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  6: onset=417.098s, val=1.409969e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  7: onset=464.092s, val=4.109977e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  8: onset=507.697s, val=1.414992e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial  9: onset=552.086s, val=1.405009e-05, code=  0, file=generated/silence_or_baseline.wav
  Trial 10: onset=597.493s, val=0.000000e+00, c

In [ ]:
# Check music channel values at 768 markers for both genMusic and classicalMusic
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd
import numpy as np

print("🔍 MUSIC CHANNEL VALUES AT 768 TRIAL MARKERS")
print("=" * 55)

tasks_to_check = ['genMusic01', 'classicalMusic']

for task in tasks_to_check:
    print(f"\n--- {task.upper()} ---")
    
    # Load data
    bids_path = BIDSPath(
        subject='01',
        task=task,
        datatype='eeg',
        root='./datasets/bcmi/bcmi-fmri'
    )
    
    try:
        raw = read_raw_bids(bids_path, verbose=False)
        events_path = bids_path.copy().update(suffix='events', extension='.tsv')
        events_df = pd.read_csv(events_path.fpath, sep='\t')
        
        # Find 768 events
        trial_starts = events_df[events_df['trial_type'] == 768].copy()
        print(f"Total 768 events: {len(trial_starts)}")
        
        if len(trial_starts) > 0 and 'music' in raw.ch_names:
            music_idx = raw.ch_names.index('music')
            sfreq = raw.info['sfreq']
            
            print(f"Music channel values at 768 markers:")
            for i, (_, event) in enumerate(trial_starts.head(10).iterrows()):  # Check first 10
                onset = event['onset']
                sample_idx = int(onset * sfreq)
                
                if sample_idx < raw.n_times:
                    music_val = raw.get_data(picks=[music_idx])[0, sample_idx]
                    
                    # Check values shortly after the marker too
                    after_samples = [sample_idx + offset for offset in [100, 500, 1000]]  # 0.1s, 0.5s, 1s after
                    after_values = []
                    
                    for after_idx in after_samples:
                        if after_idx < raw.n_times:
                            after_val = raw.get_data(picks=[music_idx])[0, after_idx]
                            after_values.append(after_val)
                        else:
                            after_values.append(None)
                    
                    print(f"  Event {i+1}: onset={onset:7.3f}s")
                    print(f"    At marker:    {music_val:.8e}")
                    print(f"    +0.1s:        {after_values[0]:.8e}" if after_values[0] is not None else "    +0.1s:        N/A")
                    print(f"    +0.5s:        {after_values[1]:.8e}" if after_values[1] is not None else "    +0.5s:        N/A")
                    print(f"    +1.0s:        {after_values[2]:.8e}" if after_values[2] is not None else "    +1.0s:        N/A")
                    
                    # Check if value is essentially zero
                    is_zero = abs(music_val) < 1e-10
                    print(f"    Is ~zero:     {is_zero}")
                    print()
        else:
            print("No 768 events found or no music channel")
            
    except Exception as e:
        print(f"Error loading {task}: {e}")

print("✅ Analysis completed!")

🔍 MUSIC CHANNEL VALUES AT 768 TRIAL MARKERS

--- GENMUSIC01 ---
Total 768 events: 24
Music channel values at 768 markers:
  Event 1: onset=191.151s
    At marker:    1.40500862e-05
    +0.1s:        1.40500862e-05
    +0.5s:        1.40500862e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 2: onset=191.158s
    At marker:    1.40500862e-05
    +0.1s:        1.40500862e-05
    +0.5s:        1.40500862e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False



/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of chan

  Event 3: onset=238.056s
    At marker:    2.25996918e-05
    +0.1s:        2.25996918e-05
    +0.5s:        2.25996918e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 4: onset=238.060s
    At marker:    2.25996918e-05
    +0.1s:        2.25996918e-05
    +0.5s:        2.25996918e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 5: onset=283.447s
    At marker:    1.40996910e-05
    +0.1s:        1.40996910e-05
    +0.5s:        1.40996910e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 6: onset=283.451s
    At marker:    1.40996910e-05
    +0.1s:        1.40996910e-05
    +0.5s:        1.40996910e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 7: onset=327.102s
    At marker:    4.10501625e-05
    +0.1s:        4.10501625e-05
    +0.5s:        4.10501625e-05
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 8: onset=327.106s
    At marker:    4.10501625e-05
    +0.1s:        4.105016

/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  raw = read_raw_bids(bids_path, verbose=False)
/tmp/ipykernel_26767/3196285421.py:23: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of chan

Total 768 events: 12
Music channel values at 768 markers:
  Event 1: onset= 31.771s
    At marker:    3.50000000e-07
    +0.1s:        3.50000000e-07
    +0.5s:        0.00000000e+00
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 2: onset=195.807s
    At marker:    2.50001526e-07
    +0.1s:        2.50001526e-07
    +0.5s:        0.00000000e+00
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 3: onset=362.010s
    At marker:    1.49997711e-07
    +0.1s:        1.49997711e-07
    +0.5s:        1.49997711e-07
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 4: onset=517.864s
    At marker:    1.49997711e-07
    +0.1s:        1.49997711e-07
    +0.5s:        0.00000000e+00
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 5: onset=676.140s
    At marker:    2.50001526e-07
    +0.1s:        2.50001526e-07
    +0.5s:        2.50001526e-07
    +1.0s:        0.00000000e+00
    Is ~zero:     False

  Event 6: onset=841.794s
 

In [ ]:
print("🔬 TESTING DIFFERENT SCALING APPROACHES")
print("=" * 50)

# Sample values from our analysis
genmusic_values = [1.40500862e-05, 2.25996918e-05, 1.40996910e-05, 4.10501625e-05]
classical_values = [3.50000000e-07, 2.50001526e-07, 1.49997711e-07]

print("\n--- GENMUSIC VALUES ---")
for i, val in enumerate(genmusic_values[:4]):
    print(f"Original value: {val:.2e}")
    print(f"  × 20:        {val * 20:.6f}")
    print(f"  × 1000:      {val * 1000:.6f}")
    print(f"  × 100000:    {val * 100000:.1f}")
    print(f"  × 1000000:   {val * 1000000:.1f}")
    # Try to find integer codes
    for scale in [20, 1000, 100000, 1000000]:
        scaled = val * scale
        if 0 < scaled < 1000:  # reasonable music file range
            print(f"  → {scale:>7}× = {scaled:.0f} (music file code?)")
    print()

print("\n--- CLASSICAL VALUES ---")
for i, val in enumerate(classical_values[:3]):
    print(f"Original value: {val:.2e}")
    print(f"  × 20:        {val * 20:.6f}")
    print(f"  × 1000:      {val * 1000:.6f}")
    print(f"  × 100000:    {val * 100000:.6f}")
    print(f"  × 1000000:   {val * 1000000:.1f}")
    print(f"  × 10000000:  {val * 10000000:.1f}")
    # Try to find integer codes
    for scale in [20, 1000, 100000, 1000000, 10000000]:
        scaled = val * scale
        if 0 < scaled < 1000:  # reasonable music file range
            print(f"  → {scale:>8}× = {scaled:.0f} (music file code?)")
    print()

🔬 TESTING DIFFERENT SCALING APPROACHES

--- GENMUSIC VALUES ---
Original value: 1.41e-05
  × 20:        0.000281
  × 1000:      0.014050
  × 100000:    1.4
  × 1000000:   14.1
  →      20× = 0 (music file code?)
  →    1000× = 0 (music file code?)
  →  100000× = 1 (music file code?)
  → 1000000× = 14 (music file code?)

Original value: 2.26e-05
  × 20:        0.000452
  × 1000:      0.022600
  × 100000:    2.3
  × 1000000:   22.6
  →      20× = 0 (music file code?)
  →    1000× = 0 (music file code?)
  →  100000× = 2 (music file code?)
  → 1000000× = 23 (music file code?)

Original value: 1.41e-05
  × 20:        0.000282
  × 1000:      0.014100
  × 100000:    1.4
  × 1000000:   14.1
  →      20× = 0 (music file code?)
  →    1000× = 0 (music file code?)
  →  100000× = 1 (music file code?)
  → 1000000× = 14 (music file code?)

Original value: 4.11e-05
  × 20:        0.000821
  × 1000:      0.041050
  × 100000:    4.1
  × 1000000:   41.1
  →      20× = 0 (music file code?)
  →    1000× =

In [ ]:
print("🎵 COMPREHENSIVE MUSIC FILE MAPPING TEST")
print("=" * 50)

import os
from pathlib import Path

# Check what music files exist
stimuli_path = Path("/home/zmrocze/studia/uwr/magisterka/datasets/bcmi/bcmi-fmri/stimuli")
generated_files = list((stimuli_path / "generated").glob("*.wav"))
classical_files = list((stimuli_path / "classical").glob("*.mp3"))

print(f"📁 Found {len(generated_files)} generated music files")
print(f"📁 Found {len(classical_files)} classical music files")

# Test our scaling hypothesis
def decode_music_channel(value, task_type):
    """Decode music channel value to get file identifier"""
    if task_type == "genMusic":
        # Scale by 100,000 for generated music
        scaled = value * 100000
        return int(round(scaled))
    elif task_type == "classicalMusic":
        # Scale by 10,000,000 for classical music  
        scaled = value * 10000000
        return int(round(scaled))
    return None

# Test with our sample values
print("\n--- GENMUSIC DECODING ---")
genmusic_values = [1.40500862e-05, 2.25996918e-05, 1.40996910e-05, 4.10501625e-05]
for i, val in enumerate(genmusic_values):
    code = decode_music_channel(val, "genMusic")
    print(f"Value {val:.2e} → Code {code}")
    
    # Try to find matching files
    matches = [f for f in generated_files if f.name.startswith(f"{code}-")]
    if matches:
        print(f"  ✅ Found {len(matches)} files: {matches[0].name} ...")
    else:
        print(f"  ❌ No files found for code {code}")

print("\n--- CLASSICAL DECODING ---")
classical_values = [3.50000000e-07, 2.50001526e-07, 1.49997711e-07]
for i, val in enumerate(classical_values):
    code = decode_music_channel(val, "classicalMusic")
    print(f"Value {val:.2e} → Code {code}")
    
    # Try to find matching files
    matches = [f for f in classical_files if f.name.startswith(f"p{code}_")]
    if matches:
        print(f"  ✅ Found file: {matches[0].name}")
    else:
        print(f"  ❌ No file found for code p{code}")

# Let's also list the actual classical files to see the pattern
print("\n--- CLASSICAL FILES ---")
for f in sorted(classical_files):
    print(f"  {f.name}")

🎵 COMPREHENSIVE MUSIC FILE MAPPING TEST
📁 Found 216 generated music files
📁 Found 7 classical music files

--- GENMUSIC DECODING ---
Value 1.41e-05 → Code 1
  ✅ Found 24 files: 1-2_3.wav ...
Value 2.26e-05 → Code 2
  ✅ Found 24 files: 2-9_1.wav ...
Value 1.41e-05 → Code 1
  ✅ Found 24 files: 1-2_3.wav ...
Value 4.11e-05 → Code 4
  ✅ Found 24 files: 4-1_3.wav ...

--- CLASSICAL DECODING ---
Value 3.50e-07 → Code 4
  ✅ Found file: p4_chopin-n4-op28-Konczal.mp3
Value 2.50e-07 → Code 3
  ✅ Found file: p3_chopin-n3-op10-bertoglio.mp3
Value 1.50e-07 → Code 1
  ✅ Found file: p1_chopin-n10-op12-bertoglio.mp3

--- CLASSICAL FILES ---
  p1_chopin-n10-op12-bertoglio.mp3
  p2_rachmaninoff-n5-op32-prelude-in-g-major.mp3
  p3_chopin-n3-op10-bertoglio.mp3
  p4_chopin-n4-op28-Konczal.mp3
  p5_mendelssohn-variations-serieuses-op54-larrard.mp3
  p6_beethoven-n4-allegro-molte-e-con-brio-sinadinovic.mp3
  p7_beethoven-n3-adagio-con-expression-bertoli.mp3


In [ ]:
print("🧪 TESTING CORRECTED BCMIFMRILoader")
print("=" * 50)

# Reload the module to get the updated implementation
import importlib
import bcmi
importlib.reload(bcmi)

from bcmi import BCMIFMRILoader

# Test with our dataset
loader = BCMIFMRILoader("/home/zmrocze/studia/uwr/magisterka/datasets/bcmi/bcmi-fmri")

# Load subjects using the correct method
print("📁 Loading subjects...")
try:
    loader.load_all_subjects(max_subjects=1, max_runs_per_session=2)  # Load just one subject with 2 runs to test
    
    if loader.data:
        print(f"✅ Data loaded successfully!")
        print(f"   Available subjects: {list(loader.data.keys())}")
        
        # Check the processed events for each subject
        for subject_id, subject_data in loader.data.items():
            print(f"\n=== SUBJECT {subject_id} ===")
            for session_id, session_data in subject_data.items():
                print(f"\n--- SESSION {session_id} ---")
                for run_id, run_data in session_data.items():
                    task_name = run_data['experimental_info']['task_name']
                    processed_events = run_data.get('processed_events', {})
                    fmri_trials = processed_events.get('fmri_trials', [])
                    
                    print(f"\n--- RUN {run_id} ({task_name}) ---")
                    print(f"Found {len(fmri_trials)} fMRI trials")
                    
                    for i, trial in enumerate(fmri_trials[:3]):  # Show first 3 trials
                        print(f"  Trial {trial['trial_number']}: {trial['onset']:.1f}s")
                        print(f"    Task: {trial['task_type']}")
                        print(f"    Music: {trial['music_file']}")
                        print(f"    Code: {trial['music_code']}")
                        print(f"    Duration: {trial['duration']:.1f}s")
                    
                    if len(fmri_trials) > 3:
                        print(f"  ... and {len(fmri_trials) - 3} more trials")
        
    else:
        print("❌ No data loaded")
        
except Exception as e:
    print(f"❌ Error loading data: {e}")
    import traceback
    traceback.print_exc()

🧪 TESTING CORRECTED BCMIFMRILoader
📁 Loading subjects...
🔄 Loading bcmi-fmri dataset...
📦 Subjects to load: 1 of 21
🏃 Max runs per session: 2

Loading subject 01 (bcmi-fmri):
  ✓ Run classicalMusic: 1915.0s, 2256 events, 1 trials


/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_valance" will be set to "misc".
  
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_arousal" will be set to "misc".
  
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_x" will be set to "misc".
  
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No BIDS -> MNE mapping found for channel type "reponse". Type of channel "ft_y" will be set to "misc".
  
/home/zmrocze/studia/uwr/magisterka/src/bcmi.py:299: RuntimeWarning: No B

  ✓ Run genMusic01: 742.0s, 1162 events, 1 trials

📊 LOADING SUMMARY:
• Successfully loaded: 1 subjects
• Failed to load: 0 subjects
• Success rate: 100.0%
✅ Data loaded successfully!
   Available subjects: ['01']

=== SUBJECT 01 ===

--- SESSION no_session ---

--- RUN classicalMusic (classicalMusic) ---
Found 12 fMRI trials
  Trial 1: 31.8s
    Task: classicalMusic
    Music: classical/p4_*.mp3
    Code: 4
    Duration: 164.0s
  Trial 2: 195.8s
    Task: classicalMusic
    Music: classical/p3_*.mp3
    Code: 3
    Duration: 166.2s
  Trial 3: 362.0s
    Task: classicalMusic
    Music: classical/p1_*.mp3
    Code: 1
    Duration: 155.9s
  ... and 9 more trials

--- RUN genMusic01 (genMusic01) ---
Found 12 fMRI trials
  Trial 1: 191.2s
    Task: generatedMusic
    Music: generated/1-*_*.wav
    Code: 1
    Duration: 40.0s
  Trial 2: 238.1s
    Task: generatedMusic
    Music: generated/2-*_*.wav
    Code: 2
    Duration: 40.0s
  Trial 3: 283.4s
    Task: generatedMusic
    Music: generat